In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
stores_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holi_event_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")

In [4]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [10]:
oil_data.head()


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [11]:
holi_event_data.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [6]:
train_data.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [9]:
train_data.set_index(train_data['date'])
train_data.index.names = ['date_time']
train_data.index = pd.to_datetime(train_data.index)
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
date_time,,,,,,
1970-01-01 00:00:00.000000000,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1970-01-01 00:00:00.000000001,1,2013-01-01,1,BABY CARE,0.0,0
1970-01-01 00:00:00.000000002,2,2013-01-01,1,BEAUTY,0.0,0
1970-01-01 00:00:00.000000003,3,2013-01-01,1,BEVERAGES,0.0,0
1970-01-01 00:00:00.000000004,4,2013-01-01,1,BOOKS,0.0,0


In [12]:
def split_date_time(df):
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['weekday'] = df.index.weekday
    df['year']= df.index.year
    return df


In [13]:
train_data = split_date_time(train_data)
train_data['is_weekday'] = train_data['weekday'].apply(lambda x: 1 if x>4 else 0)
train_data.head()

,id,date,store_nbr,family,sales,onpromotion,month,day,weekday,year,is_weekday
date_time,,,,,,,,,,,
1970-01-01 00:00:00.000000000,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,1,3,1970,0
1970-01-01 00:00:00.000000001,1,2013-01-01,1,BABY CARE,0.0,0,1,1,3,1970,0
1970-01-01 00:00:00.000000002,2,2013-01-01,1,BEAUTY,0.0,0,1,1,3,1970,0
1970-01-01 00:00:00.000000003,3,2013-01-01,1,BEVERAGES,0.0,0,1,1,3,1970,0
1970-01-01 00:00:00.000000004,4,2013-01-01,1,BOOKS,0.0,0,1,1,3,1970,0


In [14]:
merged_df = pd.merge(train_data,oil_data,on="date",how="left").merge(right=stores_data,on="store_nbr",how="left").merge(right=holi_event_data,on="date",how="left")
merged_df.describe()

,id,store_nbr,sales,onpromotion,month,day,weekday,year,is_weekday,dcoilwtico,cluster
count,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3054348.0,3054348.0,3054348.0,3054348.0,3054348.0,2.099196e+06,3.054348e+06
mean,1.504277e+06,2.750000e+01,3.590209e+02,2.617480e+00,1.0,1.0,3.0,1970.0,0.0,6.801587e+01,8.481481e+00
std,8.662610e+05,1.558579e+01,1.107286e+03,1.225494e+01,0.0,0.0,0.0,0.0,0.0,2.569134e+01,4.649735e+00
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,3.0,1970.0,0.0,2.619000e+01,1.000000e+00
25%,7.546768e+05,1.400000e+01,0.000000e+00,0.000000e+00,1.0,1.0,3.0,1970.0,0.0,4.641000e+01,4.000000e+00
50%,1.507572e+06,2.750000e+01,1.100000e+01,0.000000e+00,1.0,1.0,3.0,1970.0,0.0,5.343000e+01,8.500000e+00
75%,2.255120e+06,4.100000e+01,1.960110e+02,0.000000e+00,1.0,1.0,3.0,1970.0,0.0,9.581000e+01,1.300000e+01
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02,1.0,1.0,3.0,1970.0,0.0,1.106200e+02,1.700000e+01


In [15]:
merged_df.head()


,id,date,store_nbr,family,sales,onpromotion,month,day,weekday,year,...,dcoilwtico,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,1,3,1970,...,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,1,1,3,1970,...,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,1,1,3,1970,...,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,1,3,1970,...,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,1,1,3,1970,...,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   month        int32  
 7   day          int32  
 8   weekday      int32  
 9   year         int32  
 10  is_weekday   int64  
 11  dcoilwtico   float64
 12  city         object 
 13  state        object 
 14  type_x       object 
 15  cluster      int64  
 16  type_y       object 
 17  locale       object 
 18  locale_name  object 
 19  description  object 
 20  transferred  object 
dtypes: float64(2), int32(4), int64(5), object(10)
memory usage: 442.8+ MB


In [17]:
merged_df.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
month                0
day                  0
weekday              0
year                 0
is_weekday           0
dcoilwtico      955152
city                 0
state                0
type_x               0
cluster              0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dtype: int64

In [18]:
merged_df['dcoilwtico']= merged_df['dcoilwtico'].fillna(0)
merged_df.query('date=="2016-04-16"')

,id,date,store_nbr,family,sales,onpromotion,month,day,weekday,year,...,dcoilwtico,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
2161566,2134836,2016-04-16,1,AUTOMOTIVE,6.000,0,1,1,3,1970,...,0.0,Quito,Pichincha,D,13,Event,National,Ecuador,Terremoto Manabi,False
2161567,2134837,2016-04-16,1,BABY CARE,0.000,0,1,1,3,1970,...,0.0,Quito,Pichincha,D,13,Event,National,Ecuador,Terremoto Manabi,False
2161568,2134838,2016-04-16,1,BEAUTY,4.000,0,1,1,3,1970,...,0.0,Quito,Pichincha,D,13,Event,National,Ecuador,Terremoto Manabi,False
2161569,2134839,2016-04-16,1,BEVERAGES,2073.000,5,1,1,3,1970,...,0.0,Quito,Pichincha,D,13,Event,National,Ecuador,Terremoto Manabi,False
2161570,2134840,2016-04-16,1,BOOKS,0.000,0,1,1,3,1970,...,0.0,Quito,Pichincha,D,13,Event,National,Ecuador,Terremoto Manabi,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163343,2136613,2016-04-16,9,POULTRY,668.427,2,1,1,3,1970,...,0.0,Quito,Pichincha,B,6,Event,National,Ecuador,Terremoto Manabi,False
2163344,2136614,2016-04-16,9,PREPARED FOODS,135.720,0,1,1,3,1970,...,0.0,Quito,Pichincha,B,6,Event,National,Ecuador,Terremoto Manabi,False
2163345,2136615,2016-04-16,9,PRODUCE,1496.493,0,1,1,3,1970,...,0.0,Quito,Pichincha,B,6,Event,National,Ecuador,Terremoto Manabi,False
2163346,2136616,2016-04-16,9,SCHOOL AND OFFICE SUPPLIES,2.000,0,1,1,3,1970,...,0.0,Quito,Pichincha,B,6,Event,National,Ecuador,Terremoto Manabi,False


In [19]:
merged_df = merged_df.fillna("No holiday")
merged_df.loc[merged_df.transferred == True,"type_y"] = "No holiday"

In [20]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   month        int32  
 7   day          int32  
 8   weekday      int32  
 9   year         int32  
 10  is_weekday   int64  
 11  dcoilwtico   float64
 12  city         object 
 13  state        object 
 14  type_x       object 
 15  cluster      int64  
 16  type_y       object 
 17  locale       object 
 18  locale_name  object 
 19  description  object 
 20  transferred  object 
dtypes: float64(2), int32(4), int64(5), object(10)
memory usage: 442.8+ MB


In [21]:
merged_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,3054348.0,1.504277e+06,866261.012008,0.0,754676.75,1507571.50,2255120.250,3000887.00
store_nbr,3054348.0,2.750000e+01,15.585787,1.0,14.00,27.50,41.000,54.00
sales,3054348.0,3.590209e+02,1107.285897,0.0,0.00,11.00,196.011,124717.00
onpromotion,3054348.0,2.617480e+00,12.254939,0.0,0.00,0.00,0.000,741.00
month,3054348.0,1.000000e+00,0.000000,1.0,1.00,1.00,1.000,1.00
day,3054348.0,1.000000e+00,0.000000,1.0,1.00,1.00,1.000,1.00
weekday,3054348.0,3.000000e+00,0.000000,3.0,3.00,3.00,3.000,3.00
year,3054348.0,1.970000e+03,0.000000,1970.0,1970.00,1970.00,1970.000,1970.00
is_weekday,3054348.0,0.000000e+00,0.000000,0.0,0.00,0.00,0.000,0.00
dcoilwtico,3054348.0,4.674603e+01,38.051512,0.0,0.00,47.02,91.530,110.62


In [22]:
merged_df.query('type_y=="No holiday"')

,id,date,store_nbr,family,sales,onpromotion,month,day,weekday,year,...,dcoilwtico,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
1782,1782,2013-01-02,1,AUTOMOTIVE,2.000,0,1,1,3,1970,...,93.14,Quito,Pichincha,D,13,No holiday,No holiday,No holiday,No holiday,No holiday
1783,1783,2013-01-02,1,BABY CARE,0.000,0,1,1,3,1970,...,93.14,Quito,Pichincha,D,13,No holiday,No holiday,No holiday,No holiday,No holiday
1784,1784,2013-01-02,1,BEAUTY,2.000,0,1,1,3,1970,...,93.14,Quito,Pichincha,D,13,No holiday,No holiday,No holiday,No holiday,No holiday
1785,1785,2013-01-02,1,BEVERAGES,1091.000,0,1,1,3,1970,...,93.14,Quito,Pichincha,D,13,No holiday,No holiday,No holiday,No holiday,No holiday
1786,1786,2013-01-02,1,BOOKS,0.000,0,1,1,3,1970,...,93.14,Quito,Pichincha,D,13,No holiday,No holiday,No holiday,No holiday,No holiday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052561,2999101,2017-08-14,9,POULTRY,283.429,0,1,1,3,1970,...,47.59,Quito,Pichincha,B,6,No holiday,No holiday,No holiday,No holiday,No holiday
3052562,2999102,2017-08-14,9,PREPARED FOODS,114.120,0,1,1,3,1970,...,47.59,Quito,Pichincha,B,6,No holiday,No holiday,No holiday,No holiday,No holiday
3052563,2999103,2017-08-14,9,PRODUCE,1348.425,7,1,1,3,1970,...,47.59,Quito,Pichincha,B,6,No holiday,No holiday,No holiday,No holiday,No holiday
3052564,2999104,2017-08-14,9,SCHOOL AND OFFICE SUPPLIES,182.000,11,1,1,3,1970,...,47.59,Quito,Pichincha,B,6,No holiday,No holiday,No holiday,No holiday,No holiday


In [23]:
enc = OrdinalEncoder()

NameError: name 'OrdinalEncoder' is not defined